In [1]:
import xml.etree.ElementTree as ET
import json

def extract_param_props(param_element):
    props = {}
    for item in param_element.findall('./items/item'):
        name_attr = item.get('name')
        if name_attr == 'Name':
            props['name'] = item.text
        elif name_attr == 'NickName':
            props['nickname'] = item.text
        elif name_attr == 'Description':
            props['description'] = item.text
    # Infer kind from Name if not directly available, otherwise use a placeholder or specific logic
    props['kind'] = props.get('name', 'unknown') # Using name as kind for now
    return props

def parse_components_xml(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    components = []

    definition_objects_chunk = root.find("./chunks/chunk[@name='Definition']/chunks/chunk[@name='DefinitionObjects']")
    if definition_objects_chunk is None:
        return components

    for obj_chunk in definition_objects_chunk.findall("./chunks/chunk[@name='Object']"):
        component_props = {}

        # Extract top-level component info
        for item in obj_chunk.findall('./items/item'):
            if item.get('name') == 'Name':
                component_props['name'] = item.text

        container_chunk = obj_chunk.find("./chunks/chunk[@name='Container']")
        if container_chunk is not None:
            for item in container_chunk.findall('./items/item'):
                name_attr = item.get('name')
                if name_attr == 'NickName':
                    component_props['nickname'] = item.text
                elif name_attr == 'Description':
                    component_props['description'] = item.text

            # Extract Inputs
            inputs = []
            # Try normal structure first for inputs
            input_param_chunks_normal = container_chunk.findall("./chunks/chunk[@name='param_input']")
            if input_param_chunks_normal:
                for param_chunk in input_param_chunks_normal:
                    inputs.append(extract_param_props(param_chunk))
            else:
                # Try cluster structure for inputs
                parameter_data_chunk = container_chunk.find("./chunks/chunk[@name='ParameterData']")
                if parameter_data_chunk is not None:
                    for param_chunk in parameter_data_chunk.findall("./chunks/chunk[@name='InputParam']"):
                        inputs.append(extract_param_props(param_chunk))
            if inputs:
                component_props['inputs'] = inputs

            # Extract Outputs
            outputs = []
            # Try normal structure first for outputs
            output_param_chunks_normal = container_chunk.findall("./chunks/chunk[@name='param_output']")
            if output_param_chunks_normal:
                for param_chunk in output_param_chunks_normal:
                    outputs.append(extract_param_props(param_chunk))
            else:
                # Try cluster structure for outputs
                # Check for parameter_data_chunk again or ensure it's available if not found for inputs
                parameter_data_chunk = container_chunk.find("./chunks/chunk[@name='ParameterData']")
                if parameter_data_chunk is not None:
                    for param_chunk in parameter_data_chunk.findall("./chunks/chunk[@name='OutputParam']"):
                        outputs.append(extract_param_props(param_chunk))
            if outputs:
                component_props['outputs'] = outputs

        if component_props.get('name'): # Ensure component has a name
            components.append(component_props)

    return components

# Path to your XML file
# This file should now be the one that contains mixed component types, e.g., components.xml
xml_file = r"..\groups\modeling.ghx"
extracted_data = parse_components_xml(xml_file)

# Output as JSON
json_output = json.dumps(extracted_data, indent=4)
print(json_output)

# You can also save this to a file:
with open(    r"..\groups\modeling.json", "w") as f:  # Changed output filename to avoid overwriting
    json.dump(extracted_data, f, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: '..\\groups\\modeling.ghx'